In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import prophet

#city indexes: Adilabad->0,Karimnagar->1,Khammam->2,Nizamabad->3,Warangal->4

if(index==0):
    Df=pd.read_csv("Adilabad/city_combined.csv")
elif(index==1):
    Df=pd.read_csv("Karimnagar/city_combined.csv")
elif(index==2):
    Df=pd.read_csv("Khammam/city_combined.csv")
elif(index==3):
    Df=pd.read_csv("Nizamabad/city_combined.csv")
elif(index==4):
    Df=pd.read_csv("Warangal/city_combined.csv")

Df.drop('temp_min (⁰C)',axis=1,inplace=True)


Df['windspeed(Kmph)']=(Df['wind_speed_max (Kmph)']+Df['wind_speed_min (Kmph)'])/2
Df['Humidity(%)']=(Df['Humidity_max (%)']+Df['humidity_min (%)'])/2
Df.drop(['wind_speed_min (Kmph)','humidity_min (%)','Humidity_max (%)','wind_speed_max (Kmph)'],axis=1,inplace=True)

Df['windspeed(Kmph)'].fillna(value=Df['windspeed(Kmph)'].mean(),inplace=True)

lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-22', 'lower_window': 0, 'ds_upper': '2020-05-31'},
    {'holiday': 'lockdown_2', 'ds': '2021-05-12', 'lower_window': 0, 'ds_upper': '2021-05-22'}])
for t_col in ['ds', 'ds_upper']:
    lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days

m=prophet.Prophet(growth='logistic',holidays=lockdowns,yearly_seasonality=True)
m.add_country_holidays(country_name='IN')
Df['ds']=Df['Date']
Df['ds'] = pd.to_datetime(Df['ds'])
Df['y']=Df['temp_max (⁰C)']
cap=Df['temp_max (⁰C)'].max()+5
Df['cap']=cap
Df.drop(["Date","temp_max (⁰C)"],axis=1,inplace=True)
for i in ['Rainfall (mm)','windspeed(Kmph)',
       'Humidity(%)']:
    m.add_regressor(i)

m.fit(Df)
future = m.make_future_dataframe(periods=365)
today=[0,25,75,0.25,7.88]
j=0
for i in ['Rainfall (mm)','windspeed(Kmph)',
       'Humidity(%)']:
    future[i]=today[j]
    j+=1
future['cap']=cap
forecast=m.predict(future)
forecast=forecast[['ds','yhat','yhat_lower','yhat_upper']]

C:\Users\aksha\AppData\Local\Temp\ipykernel_38296\2191887256.py:38: UserWarning: Parsing '13-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_38296\2191887256.py:38: UserWarning: Parsing '14-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_38296\2191887256.py:38: UserWarning: Parsing '15-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_38296\2191887256.py:38: UserWarning: Parsing '16-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  Df['ds'] = pd.to_datetime(Df['ds'])
C:\Users\aksha\AppData\Local\Temp\ipykernel_

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\aksha\anaconda3\lib\site-packages\prophet\hdays.py:259: Warning: We only support Diwali and Holi holidays from 2010 to 2030
  warnings.warn(warning_msg, Warning)
C:\Users\aksha\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aksha\anaconda3\lib\site-packages\prophet\forecaster.py:896: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
C:\Users\aksha\anaconda3\lib\site-packages\prophet\hdays.py:259: Warning: We only support Diwali and Holi holidays from 2010 to 2030
  warnings.warn(warning_msg, Warning)
C:\Users\aksha\anaconda3\lib\site-packages\prophet\forecaster.py:8

In [3]:
forecast

,ds,yhat,yhat_lower,yhat_upper
0,2018-01-01,28.572133,25.564434,31.499782
1,2018-01-02,32.435968,29.227538,35.456239
2,2018-01-03,32.629591,29.750084,35.797396
3,2018-01-04,32.457972,29.400329,35.629168
4,2018-01-05,32.533645,29.503706,35.520150
...,...,...,...,...
2094,2023-12-05,34.659577,31.588161,37.750642
2095,2023-12-06,34.581097,31.351482,37.393625
2096,2023-12-07,34.151937,31.288207,37.073110
2097,2023-12-08,33.989374,30.898011,36.965691


In [ ]:
#input today's date
#add records up until today
#make predictions on future = 31-12-2023-today's date
#heatwaves-make graph for each day of the month
#AQI-query each month and display AQI for each day(table) and classify the AQI